In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [12]:
# To add depth to our analysis we merged a file from the US housing info and IRS income data on home price rental prices.
# We did a full outer merge and then cleaned the data in Pandas.

income_rental_raw = pd.read_csv("income_rental_raw.csv")
income_rental_raw

,ID,State,StateAb,County,City,Type,Primary,ZipCode,Mean,Median,...,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2800.0,2875.0,2900.0,2895.0,2850.0,2925.0,2900.0,2900.0,2899.0,2900.0
1,6013288.0,California,CA,Los Angeles County,Walnut,City,place,91789.0,108837.0,100267.0,...,3500.0,3500.0,3500.0,3520.0,3500.0,3600.0,3607.0,3665.0,3600.0,3550.0
2,6013138.0,California,CA,Los Angeles County,South Gate,City,place,90280.0,54149.0,42954.0,...,3500.0,3500.0,3500.0,3520.0,3500.0,3600.0,3607.0,3665.0,3600.0,3550.0
3,6013038.0,California,CA,Los Angeles County,San Dimas,City,place,91773.0,91998.0,79150.0,...,3500.0,3500.0,3500.0,3520.0,3500.0,3600.0,3607.0,3665.0,3600.0,3550.0
4,6012998.0,California,CA,Los Angeles County,Lomita,City,place,90717.0,143582.0,140323.0,...,3500.0,3500.0,3500.0,3520.0,3500.0,3600.0,3607.0,3665.0,3600.0,3550.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3682,26011463.0,Michigan,MI,Iosco County,Tawas City,City,place,48763.0,46670.0,39547.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3683,13011239.0,Georgia,GA,Long County,Ludowici,City,place,31316.0,46236.0,37868.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3684,48013390.0,Texas,TX,Mason County,Mason,City,place,76856.0,35252.0,28290.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3685,48013570.0,Texas,TX,Young County,Olney,City,place,76374.0,48432.0,36772.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# A simple drop based on index and isna on ID removes missing values which leaves us with around 2800 rows. The final step is to eliminate unncessary columens and to get only the 2017 data to compare rental prices with median income.
# This dropped about 800 rows leaving us with 2840 rows that connect historical rental prices with historical data.

income_rental_raw.drop(income_rental_raw[income_rental_raw['ID'].isna()].index, inplace=True)

,State,StateAb,County,City,ZipCode,Mean,Median,Stdev,2010-01,2010-02,...,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12
0,California,CA,Los Angeles County,Walnut,91789.0,108837.0,100267.0,70014.0,NaN,NaN,...,3500.0,3500.0,3500.0,3520.0,3500.0,3600.0,3607.0,3665.0,3600.0,3550.0
1,California,CA,Los Angeles County,South Gate,90280.0,54149.0,42954.0,41327.0,NaN,NaN,...,3500.0,3500.0,3500.0,3520.0,3500.0,3600.0,3607.0,3665.0,3600.0,3550.0
2,California,CA,Los Angeles County,San Dimas,91773.0,91998.0,79150.0,65569.0,NaN,NaN,...,3500.0,3500.0,3500.0,3520.0,3500.0,3600.0,3607.0,3665.0,3600.0,3550.0
3,California,CA,Los Angeles County,Lomita,90717.0,143582.0,140323.0,84725.0,NaN,NaN,...,3500.0,3500.0,3500.0,3520.0,3500.0,3600.0,3607.0,3665.0,3600.0,3550.0
4,California,CA,Los Angeles County,Paramount,90723.0,55116.0,45286.0,41505.0,NaN,NaN,...,3500.0,3500.0,3500.0,3520.0,3500.0,3600.0,3607.0,3665.0,3600.0,3550.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2835,Michigan,MI,Iosco County,Tawas City,48763.0,46670.0,39547.0,35501.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2836,Georgia,GA,Long County,Ludowici,31316.0,46236.0,37868.0,41721.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2837,Texas,TX,Mason County,Mason,76856.0,35252.0,28290.0,30250.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2838,Texas,TX,Young County,Olney,76374.0,48432.0,36772.0,39508.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# The final step after deleting all ID rows is to drop unneccessary columns and then check to see if there are any rows with no rental data.
income_rental = income_rental_raw.drop(['ID', 'Primary', 'Type', 'RegionName', 'State-2', 'Metro', 'SizeRank', 'County-2'], axis=1).reset_index(drop=True)

# An easy way to check for missing all rental prices is to use the thresh argument in dropna.

income_rental_clean = income_rental.dropna(thresh=9)
income_rental_clean

,State,StateAb,County,City,ZipCode,Mean,Median,Stdev,2010-01,2010-02,...,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12
0,California,CA,Los Angeles County,Walnut,91789.0,108837.0,100267.0,70014.0,NaN,NaN,...,3500.0,3500.0,3500.0,3520.0,3500.0,3600.0,3607.0,3665.0,3600.0,3550.0
1,California,CA,Los Angeles County,South Gate,90280.0,54149.0,42954.0,41327.0,NaN,NaN,...,3500.0,3500.0,3500.0,3520.0,3500.0,3600.0,3607.0,3665.0,3600.0,3550.0
2,California,CA,Los Angeles County,San Dimas,91773.0,91998.0,79150.0,65569.0,NaN,NaN,...,3500.0,3500.0,3500.0,3520.0,3500.0,3600.0,3607.0,3665.0,3600.0,3550.0
3,California,CA,Los Angeles County,Lomita,90717.0,143582.0,140323.0,84725.0,NaN,NaN,...,3500.0,3500.0,3500.0,3520.0,3500.0,3600.0,3607.0,3665.0,3600.0,3550.0
4,California,CA,Los Angeles County,Paramount,90723.0,55116.0,45286.0,41505.0,NaN,NaN,...,3500.0,3500.0,3500.0,3520.0,3500.0,3600.0,3607.0,3665.0,3600.0,3550.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2132,Utah,UT,Utah County,Lehi,84043.0,86849.0,78361.0,52361.0,NaN,NaN,...,1425.0,1395.0,1375.0,1410.0,1410.0,1388.0,1345.0,1295.0,1275.0,1275.0
2133,Florida,FL,Broward County,Tamarac,33321.0,52731.0,42353.0,40749.0,NaN,NaN,...,2700.0,2650.0,2738.0,2775.0,3000.0,3000.0,3150.0,3000.0,3150.0,3395.0
2134,Florida,FL,Broward County,Oakland Park,33309.0,56688.0,45475.0,46744.0,NaN,NaN,...,2700.0,2650.0,2738.0,2775.0,3000.0,3000.0,3150.0,3000.0,3150.0,3395.0
2135,Florida,FL,Broward County,Miramar,33027.0,77682.0,65589.0,55002.0,NaN,NaN,...,2700.0,2650.0,2738.0,2775.0,3000.0,3000.0,3150.0,3000.0,3150.0,3395.0


In [19]:
# The final step is just to convert the merged file to a final db csv to complet the ETL process.

income_rental_clean.to_csv("rental_by_income.csv")